In [ ]:
%load_ext autoreload
%autoreload 2
from amftrack.pipeline.launching.run_super import (
    run_launcher,
    directory_scratch,
    directory_project,
    directory_project,
    run_parallel_stitch,
    run_parallel_transfer,
)
import os
from amftrack.util.sys import (
    get_dates_datetime,
    get_dirname,
    temp_path,
    get_data_info,
    update_plate_info,
    update_analysis_info,
    get_analysis_info,
    get_current_folders,
    get_folders_by_plate_id,
)

from time import time_ns
from amftrack.util.dbx import (
    upload_folders,
    load_dbx,
    download,
    get_dropbox_folders_prince,
)
from datetime import datetime
from amftrack.pipeline.launching.run_super import (
    run_parallel,
    directory_scratch,
    directory_project,
    run_parallel_stitch,
)
from amftrack.util.dbx import read_saved_dropbox_state, get_dropbox_folders_prince

%load_ext autoreload
%autoreload 2
%matplotlib widget

from amftrack.pipeline.launching.run_super import (
    run_launcher,
    directory_scratch,
    directory_project,
    directory_project,
    run_parallel_stitch,
    run_parallel_transfer,
)
import os
from amftrack.util.sys import (
    get_dates_datetime,
    get_dirname,
    temp_path,
    get_data_info,
    update_plate_info,
    update_analysis_info,
    get_analysis_info,
    get_current_folders,
    get_folders_by_plate_id,
)

from time import time_ns
from amftrack.util.dbx import (
    upload_folders,
    load_dbx,
    download,
    get_dropbox_folders_prince,
)
from datetime import datetime
from amftrack.pipeline.launching.run_super import (
    run_parallel,
    directory_scratch,
    directory_project,
    run_parallel_stitch,
)
from amftrack.util.dbx import read_saved_dropbox_state, get_dropbox_folders_prince
import sys
import os

from amftrack.util.sys import get_dirname, temp_path
import pandas as pd
import ast
from scipy import sparse
import scipy.io as sio
import cv2
import imageio.v2 as imageio
import numpy as np
import scipy.sparse
import os
from time import time
from amftrack.pipeline.functions.image_processing.extract_skel import (
    extract_skel_new_prince,
    run_back_sub,
    bowler_hat,
)

from amftrack.util.sys import get_dates_datetime, get_dirname
import shutil
import matplotlib.pyplot as plt
from amftrack.pipeline.functions.image_processing.experiment_class_surf import (
    Experiment,
    save_graphs,
    load_graphs,
)
from amftrack.pipeline.functions.image_processing.experiment_util import (
    get_random_edge,
    distance_point_edge,
    plot_edge,
    plot_edge_cropped,
    find_nearest_edge,
    get_edge_from_node_labels,
    plot_full_image_with_features,
    get_all_edges,
    get_all_nodes,
    find_neighboring_edges,
    reconstruct_image,
    reconstruct_skeletton_from_edges,
    reconstruct_skeletton_unicolor,
    reconstruct_image_from_general,
    plot_full,
)

In [ ]:
dir_drop = "/DATA/MYRISTATE/PRINCE"
all_folders_drop = get_dropbox_folders_prince(dir_drop)
# all_folders_drop = read_saved_dropbox_state("/DATA/PRINCE")

In [ ]:
plates = list(all_folders_drop["unique_id"].unique())
plates

In [ ]:
folders_drop = all_folders_drop.loc[~all_folders_drop["unique_id"].isin(plates)]

In [ ]:
folders_drop = all_folders_drop

In [ ]:
folders_drop = all_folders_drop.loc[
    all_folders_drop["unique_id"].isin(["faulty_param_files"])
]

In [ ]:
folders_drop

In [ ]:
import shutil

directory_targ = os.path.join(directory_scratch, "stitch_temp2") + "/"

for folder in folders_drop["folder"]:
    # print(os.path.join(directory_targ,folder))
    current_path = os.path.join(directory_targ, folder, "Img", "Img")
    new_folder_path = os.path.join(directory_targ, folder, "Img2")
    new_folder_path2 = os.path.join(directory_targ, folder, "Img")

    if os.path.exists(current_path):
        # if os.path.exists(os.path.join(directory_targ,folder)):
        shutil.move(current_path, new_folder_path)
        shutil.rmtree(os.path.join(directory_targ, folder, "Img"))
        shutil.move(new_folder_path, new_folder_path2)

        print(folder)
        # shutil.rmtree(os.path.join(directory_targ,folder))
        # break
        # break

In [ ]:
len(folders_drop)

In [ ]:
name_job = "Myr"
directory_targ = directory_project


run_parallel_transfer(
    "from_drop.py",
    [directory_targ],
    folders_drop,
    50,
    "4:00:00",
    "staging",
    cpus=1,
    node="staging",
    name_job=name_job,
)

In [ ]:
plates = list(all_folders_drop["unique_id"])

In [ ]:
directory_targ = directory_project
update_plate_info(directory_targ, local=True)
all_folders = get_current_folders(directory_targ, local=True)

In [ ]:
folders = all_folders.loc[all_folders["unique_id"].isin(plates)]
print(len(folders))
folders = folders[folders["/Analysis/skeleton_realigned_compressed.mat"] == False]
print(len(folders))

In [ ]:
folders["unique_id"].unique()

In [ ]:
folders = all_folders.loc[all_folders["medium"].str.contains("Myr")]
folders = folders.sort_values(by="datetime")

len(folders)

In [ ]:
# folders = all_folders.loc[all_folders["unique_id"].isin(plates)]
# dir_drop = "DATA/PRINCE"
dir_drop = "DATA/MYRISTATE/PRINCE"
name_job = "load_back"
delete = True
run_parallel_transfer(
    "toward_drop.py",
    [dir_drop, delete],
    folders,
    50,
    "4:00:00",
    "staging",
    cpus=1,
    node="staging",
    name_job=name_job,
)

In [ ]:
folders = all_folders.loc[all_folders["medium"].str.contains("Myr")]

folders = folders.loc[folders["/Analysis/nx_graph_pruned.p"] == True]
len(folders)

In [ ]:
folders = all_folders.loc[all_folders["unique_id"].isin(plates)]
folders = folders.loc[folders["/Analysis/skeleton_realigned_compressed.mat"] == True]
num_parallel = 128
time = "2:00:00"
skip = False
resolution = 10
args = [directory_targ, skip, resolution]
run_parallel(
    "extract_width.py",
    args,
    folders,
    num_parallel,
    time,
    "extract_width2",
    cpus=128,
    node="fat",
    name_job=name_job,
)

In [ ]:
name_job = "fixing"
num_parallel = 128
time = "30:00"
args = [directory_targ]
run_parallel(
    "extract_nx_graph.py",
    args,
    folders,
    num_parallel,
    time,
    "extract_nx",
    cpus=128,
    node="fat",
    name_job=name_job,
)

In [ ]:
dir_drop = "DATA/MYRISTATE/Network_extraction"

delete = True
run_parallel_transfer(
    "toward_drop.py",
    [dir_drop, delete],
    folders,
    50,
    "4:00:00",
    "staging",
    cpus=1,
    node="staging",
    name_job="hannah",
)

In [ ]:
from amftrack.pipeline.functions.image_processing.experiment_class_surf import (
    Experiment,
    save_graphs,
    load_graphs,
)
from amftrack.pipeline.functions.image_processing.experiment_util import (
    get_random_edge,
    distance_point_edge,
    plot_edge,
    plot_edge_cropped,
    find_nearest_edge,
    get_edge_from_node_labels,
    plot_full_image_with_features,
    get_all_edges,
    get_all_nodes,
    find_neighboring_edges,
    reconstruct_image,
    reconstruct_skeletton_from_edges,
    reconstruct_skeletton_unicolor,
    reconstruct_image_from_general,
    plot_full,
)

In [ ]:
exp = Experiment(directory_targ)
exp.load(folders.iloc[1:2], suffix="")
exp.dates.sort()
for t in range(exp.ts):
    exp.load_tile_information(t)

for t in range(exp.ts):
    exp.load_tile_information(t)

load_graphs(exp, directory_targ, post_process=False, suffix="")
# import networkx as nx
# G = exp.nx_graph[t]
# largest_cc = max(nx.connected_components(G), key=len)
# exp.nx_graph[t] = G.subgraph(largest_cc).copy()

In [ ]:
import numpy as np
import matplotlib.pyplot as plt

edges = get_all_edges(exp, t)

lengths = [edge.length_um(0) for edge in edges]
fig, ax = plt.subplots()
ax.hist(lengths, 50, weights=lengths)
ax.set_yscale("log")

In [ ]:
import numpy as np
import matplotlib.pyplot as plt

widths = [edge.width(0) for edge in edges]
fig, ax = plt.subplots()
ax.hist(widths, 50)
ax.set_yscale("log")

In [ ]:
import networkx as nx


def get_largest_components(graph, num_components):
    components = list(nx.connected_components(graph))
    largest_components = sorted(components, key=len, reverse=True)[:num_components]
    return [graph.subgraph(comp) for comp in largest_components]


def create_combined_graph(graph, num_components):
    largest_components = get_largest_components(graph, num_components)
    G = largest_components[0]
    for g in largest_components[1:]:
        G = nx.compose(G, g)
    return G

In [ ]:
exp.nx_graph[t] = create_combined_graph(exp.nx_graph[t], 2)

In [ ]:
%matplotlib widget
edges = get_all_edges(exp, t)

# edges_plot = [edge for edge in edges if edge.width(0) > 5]
# edges_plot = [edge for edge in edges if edge.length_um(0) > 1000]

edges_plot = edges
plot_full(
    exp,
    t,
    downsizing=5,
    edges=edges_plot,
    dilation=15,
    prettify=False,
    figsize=(8, 6),
    # dpi=390,
    node_size=1.5,
    # save_path=path,
)

In [ ]:
[edge for edge in edges_plot if edge.length_um(0) > 1000]

In [ ]:
10 * 4096 * 1.725

In [ ]:
45**2 / 2, 30**2

In [ ]:
3000 * 1.725

In [ ]:
print("length=", np.sum([edge.length_um(0) for edge in edges_plot]) / 1000, "mm")

In [ ]:
print(
    "length=",
    np.sum([edge.length_um(0) for edge in edges_plot if edge.length_um(0) > 1000])
    / 1000,
    "mm",
)

In [ ]:
1

In [ ]:
plot_full(
    exp,
    t,
    downsizing=5,
    dilation=4,
    prettify=False,
    figsize=(16, 12),
    # dpi=390,
    node_size=1.5,
    # save_path=path,
)

In [ ]:
%matplotlib widget
%load_ext autoreload
%autoreload 2
import os
import sys

sys.path.insert(0, os.getenv("HOME") + "/pycode/MscThesis/")
# sys.path.insert(0,r'C:\Users\coren\Documents\PhD\Code\AMFtrack')

import pandas as pd
from amftrack.util.sys import (
    get_dates_datetime,
    get_dirname,
    temp_path,
    get_data_info,
    update_plate_info,
    get_current_folders,
    get_folders_by_plate_id,
)


import ast
from amftrack.plotutil import plot_t_tp1
from scipy import sparse
from datetime import datetime
import pickle
import scipy.io as sio
from pymatreader import read_mat
from matplotlib import colors
import cv2
import imageio
import matplotlib.pyplot as plt
import numpy as np
from skimage.filters import frangi
from skimage import filters
from random import choice
import scipy.sparse
import os
from amftrack.pipeline.functions.image_processing.extract_graph import (
    from_sparse_to_graph,
    generate_nx_graph,
    sparse_to_doc,
)
from skimage.feature import hessian_matrix_det
from amftrack.pipeline.functions.image_processing.experiment_class_surf import (
    Experiment,
)
from amftrack.pipeline.launching.run_super import (
    run_parallel,
    directory_scratch,
    directory_project,
    run_parallel_stitch,
)

In [ ]:
from amftrack.pipeline.functions.image_processing.extract_skel import (
    extract_skel_new_prince,
    run_back_sub,
    bowler_hat,
)
import pandas as pd

directory = directory_targ
i = 1
folder_list = list(folders["folder"])
folder_list.sort()
directory_name = folder_list[i]
print(directory_name)
# run_back_sub(directory, directory_name)
path_snap = os.path.join(directory, directory_name)
path_tile = os.path.join(path_snap, "Img/TileConfiguration.txt.registered")
try:
    tileconfig = pd.read_table(
        path_tile,
        sep=";",
        skiprows=4,
        header=None,
        converters={2: ast.literal_eval},
        skipinitialspace=True,
    )
except:
    print("error_name")
    path_tile = os.path.join(path_snap, "Img/TileConfiguration.registered.txt")
    tileconfig = pd.read_table(
        path_tile,
        sep=";",
        skiprows=4,
        header=None,
        converters={2: ast.literal_eval},
        skipinitialspace=True,
    )
dirName = path_snap + "/Analysis"
try:
    os.mkdir(path_snap + "/Analysis")
    print("Directory ", dirName, " Created ")
except FileExistsError:
    print("Directory ", dirName, " already exists")
xs = [c[0] for c in tileconfig[2]]
ys = [c[1] for c in tileconfig[2]]
name = tileconfig[0][0]
imname = "/Img3/" + name.split("/")[-1]
im = imageio.imread(directory + directory_name + imname)
dim = (
    int(np.max(ys) - np.min(ys)) + max(im.shape),
    int(np.max(xs) - np.min(xs)) + max(im.shape),
)
ims = []
skel = np.zeros(dim, dtype=np.uint8)
params = [30]

In [ ]:
j = 46

for index, name in enumerate(tileconfig[0][j : j + 1]):
    # for index, name in enumerate(list_debug):
    print(name)
    imname = "/Img3/" + name.split("/")[-1]
    im = imageio.imread(directory + directory_name + imname)
    imname2 = "/Img/" + name.split("/")[-1]
    im2 = imageio.imread(directory + directory_name + imname2)
    bowled2 = bowler_hat(-im2, 32, params)
    im[bowled2 <= 0.09] = np.maximum(im[bowled2 <= 0.09], 250)
    fig, ax = plt.subplots()
    ax.imshow(-im2, cmap="Greys")
    # shape = im.shape
    # print("segmenting")
    # segmented = extract_skel_new_prince(im, [hyph_width], perc_low, perc_high)
    # # low = np.percentile(-im+255, perc_low)
    # # high = np.percentile(-im+255, perc_high)
    # # segmented = filters.apply_hysteresis_threshold(-im+255, low, high)
    # boundaries = int(tileconfig[2][index][0] - np.min(xs)), int(
    #     tileconfig[2][index][1] - np.min(ys)
    # )
    # skel[
    #     boundaries[1] : boundaries[1] + shape[0],
    #     boundaries[0] : boundaries[0] + shape[1],
    # ] += segmented

In [ ]:
from amftrack.pipeline.functions.image_processing.extract_skel import *

hyph_width = 20
perc_low = 90
perc_high = 99.5
bowled = bowler_hat(-im.astype(np.uint8), 32, [hyph_width])
filename = time_ns()
place_save = temp_path
to_smooth = np.minimum(bowled * 255, 255 - im)
# to_smooth = 255-im
imtransformed_path = f"{place_save}/{filename}.tif"
imageio.imsave(imtransformed_path, to_smooth.astype(np.uint8))
path_anis = pastis_path
args = [0.1, 7, 0.9, 10, 50]
command = [path_anis, imtransformed_path] + args
command = [str(elem) for elem in command]
print("anis filtering")
process = subprocess.run(command, cwd=place_save, stdout=subprocess.DEVNULL)
foldname = f"{filename}_ani-K{int(args[0]*10)}s{args[1]}g{int(args[2]*10)}itD{args[3]}"
imname = foldname + f"/{foldname}it{args[4]}.tif"
path_modif = place_save + "/" + imname
try:
    im3 = imageio.imread(path_modif)
except:
    im3 = to_smooth.astype(np.uint8)
print("image_reading")
# shutil.rmtree(os.path.join(place_save, foldname))
low = max(15, np.percentile(im3, perc_low))
high = max(50, np.percentile(im3, perc_high))
transformed = im3
hyst = filters.apply_hysteresis_threshold(transformed, low, high)
dilated = remove_holes(hyst)
dilated = dilated.astype(np.uint8)
connected = remove_component(dilated)

In [ ]:
high

In [ ]:
hyph_width = 20
perc_low = 90
perc_high = 99.5
minlow = 15
minhigh = 50
connected = extract_skel_new_prince(
    im, [hyph_width], perc_low, perc_high, minlow, minhigh
)

In [ ]:
fig, ax = plt.subplots(1, 3, sharex=True, sharey=True, figsize=(10, 8))
ax[0].imshow(-im2)
ax[1].imshow(im3)
ax[2].imshow(connected)

In [ ]:
_, labels = cv2.connectedComponents(connected)

# Iterate over each component and filter based on criteria
filtered_image = np.zeros_like(connected)
for component_label in range(1, labels.max() + 1):
    # Get the mask for the current component
    component_mask = np.uint8(labels == component_label)

    # Calculate the convex hull of the component
    contours, _ = cv2.findContours(
        component_mask, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE
    )
    hull = cv2.convexHull(contours[0])

    # Calculate area and long and short axes of the convex hull
    area = cv2.contourArea(hull)
    (_, _), (long_axis, short_axis), _ = cv2.fitEllipse(hull)
    if area >= 10000 and (np.sum(component_mask) / area < 0.2 or short_axis < 100):
        filtered_image += component_mask
    print(area, np.sum(component_mask) / area, short_axis)

In [ ]:
fig, ax = plt.subplots()
ax.imshow(filtered_image, cmap="Greys")

In [ ]:
name_job = "Myr"

# name_job = f"one_shot_{time_ns()}.sh"

run_launcher(
    "stitcher.py",
    [directory_targ, name_job, -1],
    plates,
    "12:00:00",
    dependency=True,
    name_job=name_job,
)

In [ ]:
directory = directory_targ

In [ ]:
# select = all_folders.loc[(all_folders['folder'].isin([ '20210929_0308_Plate21',
#  '20210929_0912_Plate21']))]
select = folders
folder_list = list(select["folder"])
folder_list.sort()
skels = []
ims = []
kernel = np.ones((5, 5), np.uint8)
itera = 10
start = 1

# start = choice(range(len(folder_list)))
finish = start + 2
for folder in folder_list[start:finish]:
    # print(folder)
    path_snap = directory + folder
    skel_info = read_mat(path_snap + "/Analysis/skeleton_realigned_compressed.mat")
    skel = skel_info["skeleton"]
    skels.append(cv2.dilate(skel.astype(np.uint8), kernel, iterations=itera))
    im = read_mat(path_snap + "/Analysis/raw_image.mat")["raw"]
    #     im = imageio.imread(path_snap+'/StitchedImage.tif')
    #     im = cv2.resize(im,(skel.shape[1],skel.shape[0]))
    #     M = np.float32([
    # 	[1, 0, 0],
    # 	[0, 1, -117]])
    #     im = cv2.warpAffine(im,M,(im.shape[1], im.shape[0]))
    ims.append(im)

In [ ]:
plt.close("all")
start = 0
finish = start + 1
for i in range(start, finish):
    plot_t_tp1([], [], None, None, skels[i], skels[i])